In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function
import lime
import lime.lime_tabular
import dill

In [60]:
df=pd.read_csv("datasetfinal.csv")
l=len(df.index)
df=df.drop('Timestamp',axis = 1)
df=df.drop('Name', axis = 1)
df=df.drop('satisfied', axis = 1)
df=df.drop('12th Stream',axis = 1)
df=df.drop('10th',axis = 1)
df=df.drop('12th',axis = 1)
df=df.drop('interest',axis = 1)
df=df.dropna()
df.sample(3)

,creative,writing,outdoorwork,future,groupwork,enjoymost,enjoyleast,clubmost,clubleast,projectliked,projectdisliked,noengineering,drawing,futurejob,Opted
2188,I am not creative,Excited! I can share my theories with the world,I love the outdoors and wish I could work outs...,Building things with moving parts,I occasionally like working with others,Biology,Language,Consulting club,Autoshop club,Building the world's most powerful supercomputer,Programming a robot that can make you dinner,Health Studies,"I am not the best, but I am not the worst",Health,BME
1386,I am somewhat creative,Excited! I can share my theories with the world,"Working outside would be okay, but only for sh...",Designing buildings,I enjoy working with others,Maths,Social Studies,Art or design club,Robotics club,Designing an Olympic village,Building the world's most powerful supercomputer,Maths,"Really good, I can draw just about anything",Construction,CIVIL
577,I am not creative,A bit apprehensive. I get overwhelmed with so ...,"Working outside would be okay, but only for sh...",Building things with moving parts,I enjoy working with others,Maths,Chemistry,Autoshop club,Hacker club,Designing a low power invertor,Optimizing the Uber Pool routes,Maths,"I am not the best, but I am not the worst",Manufacturing,MECH


In [61]:
le_create=LabelEncoder()
le_write=LabelEncoder()
le_out=LabelEncoder()
le_future=LabelEncoder()
le_group=LabelEncoder()
le_enjoymost=LabelEncoder()
le_enjoyleast=LabelEncoder()
le_clubmost=LabelEncoder()
le_clubleast=LabelEncoder()
le_projectliked=LabelEncoder()
le_projectdisliked=LabelEncoder()
le_eng=LabelEncoder()
le_job=LabelEncoder()
le_draw=LabelEncoder()
le_opted=LabelEncoder()
df["creative"]= le_create.fit_transform(df["creative"])
df["writing"]= le_write.fit_transform(df["writing"])
df["outdoorwork"]= le_out.fit_transform(df["outdoorwork"])
df["future"]= le_future.fit_transform(df["future"])
df["groupwork"]= le_group.fit_transform(df["groupwork"])
df["enjoymost"]= le_enjoymost.fit_transform(df["enjoymost"])
df["enjoyleast"]= le_enjoyleast.fit_transform(df["enjoyleast"])
df["clubmost"]= le_clubmost.fit_transform(df["clubmost"])
df["clubleast"]= le_clubleast.fit_transform(df["clubleast"])
df["projectliked"]= le_projectliked.fit_transform(df["projectliked"])
df["projectdisliked"]= le_projectdisliked.fit_transform(df["projectdisliked"])
df["noengineering"]= le_eng.fit_transform(df["noengineering"])
df["futurejob"]= le_job.fit_transform(df["futurejob"])
df["drawing"]= le_draw.fit_transform(df["drawing"])

In [62]:
feature=df.drop('Opted',axis = 1)
target=df["Opted"]
X_train, X_test, y_train, y_test = train_test_split(feature,target, test_size = 0.3)

In [67]:
svc_model = SVC(C=100, kernel='poly', degree=3, gamma='auto', probability=True)
svc_model.fit(X_train, y_train)
prediction = svc_model.predict(X_test)

In [68]:
acc = metrics.accuracy_score(y_test, prediction)
f1=metrics.f1_score(y_test, prediction,average='micro')
print("Accuracy:",acc)
print("f1_score:",f1)
from sklearn.metrics import classification_report, confusion_matrix, precision_score
print("")

 
print(classification_report(y_test, prediction))

Accuracy: 0.9958762886597938
f1_score: 0.9958762886597938

              precision    recall  f1-score   support

         BME       0.97      1.00      0.99        39
       CIVIL       1.00      1.00      1.00        94
         CSE       1.00      0.98      0.99       107
         ECE       1.00      1.00      1.00        90
         EEE       0.99      1.00      0.99        80
        MECH       1.00      1.00      1.00        75

    accuracy                           1.00       485
   macro avg       0.99      1.00      1.00       485
weighted avg       1.00      1.00      1.00       485



In [57]:
predict_fn_rf = lambda x: svc_model.predict_proba(x).astype(float)
xt = feature.values
n=pd.unique(target)
explainer = lime.lime_tabular.LimeTabularExplainer(xt,feature_names = X_train.columns,mode="classification",class_names=['BME','CIVIL','CSE','ECE','EEE','MECH'])

In [58]:
data = {"model": svc_model,"le_create":le_create,"exp":explainer,"le_write":le_write,"le_out":le_out, "le_future": le_future, "le_group": le_group,"le_enjoymost":le_enjoymost,"le_enjyleast":le_enjoyleast,"le_clubmost":le_clubmost,"le_clubleast":le_clubleast,"le_projectliked":le_projectliked,"le_projectdisliked":le_projectdisliked,"le_eng":le_eng,"le_job":le_job,"le_draw":le_draw}
with open('svc_model1.pkl', 'wb') as file:
    dill.dump(data, file)